# Chess algorithms evaluation

This section presents the findings from the evaluation of the implemented chess algorithms and their performances. The analysis focuses on computational efficiency, move quality, and gameplay outcomes.

1. Computational efficiency

Without using GPU, this notebook takes about 300 minutes to run!!!

In [1]:
# system info
import os
import platform
import psutil  # To access more system-related info

import minimax_alphabeta
import chess
import time
import pandas as pd

import mcts

def print_system_info():
    print("="*40)
    print("System Information")
    print("="*40)
    
    # OS details
    print(f"Operating System: {platform.system()} {platform.release()}")
    print(f"Architecture: {platform.architecture()[0]}")
    
    # CPU details
    print(f"Processor: {platform.processor()}")
    print(f"Logical CPUs: {os.cpu_count()} cores")
    
    # Memory details
    if psutil:
        total_memory = round(psutil.virtual_memory().total / 1e+9, 2)  # Convert bytes to GB
        print(f"Total System Memory: {total_memory} GB")
        
        # Disk details
        total_disk_space = round(psutil.disk_usage('/').total / 1e+9, 2)  # Convert bytes to GB
        print(f"Total Disk Space: {total_disk_space} GB")
    else:
        print("psutil module is not installed, cannot fetch memory/disk info.")

    # Python version
    print(f"Python Version: {platform.python_version()}")
    print("="*40)

# Call the function to print system information
print_system_info()


System Information
Operating System: Windows 10
Architecture: 64bit
Processor: AMD64 Family 25 Model 80 Stepping 0, AuthenticAMD
Logical CPUs: 16 cores
Total System Memory: 25.13 GB
Total Disk Space: 487.2 GB
Python Version: 3.9.7


In [2]:
# minimax

fen_list = [
    "rnbqkbnr/pppppppp/8/8/8/8/PPPPPPPP/RNBQKBNR w KQkq - 0 1",  # Starting Position
    "rnb1kbnr/pppppppp/8/8/4P3/8/PPPP1qPP/RNBQKBNR w KQkq - 0 3",  # Fool's Mate
    "r1bqkb1r/pppppppp/2n5/4P3/4Q3/8/PPPP1PPP/RNB1KBNR b KQkq - 0 4",  # Scholar's Mate
    "8/8/8/8/8/4k3/4P3/4K3 w - - 0 1",  # Endgame Position (King and Pawn vs. King)
    "r1bq1rk1/ppp1bppp/2n2n2/3pp3/2PP4/2N2N2/PP2BPPP/R1BQ1RK1 w - - 0 9",  # Complex Middle Game
    "r3k2r/pppqppbp/2np1np1/4P3/3P4/2N2N2/PPP2PPP/R1BQ1RK1 w kq - 0 10",  # Position with Castling Rights
    "rnbqkbnr/ppp2ppp/8/3pP3/8/8/PPPP1PPP/RNBQKBNR b KQkq e6 0 4",  # En Passant Scenario
    "7k/5Q2/6K1/8/8/8/8/8 b - - 0 1",  # Stalemate Position
    "6k1/5ppp/8/8/8/8/5PPP/6KQ b - - 0 1",  # Checkmate Position
    "8/P7/8/8/8/8/2k5/K7 w - - 0 1"  # Underpromotion Possibility
]

fen_names = [
    "starting_position",
    "fools_mate",
    "scholars_mate",
    "endgame_king_pawn_vs_king",
    "complex_middle_game",
    "position_with_castling_rights",
    "en_passant_scenario",
    "stalemate_position",
    "checkmate_position",
    "underpromotion_possibility"
]


df_minimax = pd.DataFrame({'FEN': fen_names})

for param in range (1, 5):
    lst = []
    for fen in fen_list:
        board = chess.Board(fen)
        start_time = time.time()
        minimax_alphabeta.findMoveMinimax(board=board, depth=param)[1]
        execution_time = time.time() - start_time
        lst.append(execution_time)
    df_minimax[f'Depth={param}'] = lst


print('Minimax evaluation') # 6mins
df_minimax

Minimax evaluation


,FEN,Depth=1,Depth=2,Depth=3,Depth=4
0,starting_position,0.002038,0.029063,0.617278,13.975414
1,fools_mate,0.000000,0.001432,0.049533,1.116540
2,scholars_mate,0.001999,0.058207,1.231265,51.034503
3,endgame_king_pawn_vs_king,0.000000,0.000000,0.000000,0.015512
4,complex_middle_game,0.003000,0.083248,3.213866,110.644140
5,position_with_castling_rights,0.002001,0.090879,3.019773,110.838675
6,en_passant_scenario,0.002999,0.080670,2.770791,82.811656
7,stalemate_position,0.000000,0.000000,0.000000,0.000000
8,checkmate_position,0.000000,0.004000,0.022257,0.204414
9,underpromotion_possibility,0.000000,0.001010,0.006836,0.062989


In [3]:
# alphabeta
df_alphabeta = pd.DataFrame({'FEN': fen_names})

for param in range (1, 6):
    lst = []
    for fen in fen_list:
        board = chess.Board(fen)
        start_time = time.time()
        minimax_alphabeta.findMoveAlphaBeta(board=board, depth=param, alpha=-float('inf'), beta=float('inf'))[1]
        execution_time = time.time() - start_time
        lst.append(execution_time)
    df_alphabeta[f'Depth={param}'] = lst


print('Alpha-beta prunning evaluation') # 2mins
df_alphabeta

Alpha-beta prunning evaluation


,FEN,Depth=1,Depth=2,Depth=3,Depth=4,Depth=5
0,starting_position,0.000000,0.000000,0.062499,0.424204,3.783650
1,fools_mate,0.000000,0.000000,0.016014,0.095323,1.335317
2,scholars_mate,0.000000,0.031248,0.204314,2.171163,17.208123
3,endgame_king_pawn_vs_king,0.000000,0.000000,0.000000,0.000000,0.000000
4,complex_middle_game,0.000000,0.015625,0.204552,2.465660,16.784389
5,position_with_castling_rights,0.000000,0.031638,0.564637,4.759989,42.804841
6,en_passant_scenario,0.015627,0.015632,0.345799,2.761323,40.156484
7,stalemate_position,0.000000,0.000000,0.000000,0.000000,0.000000
8,checkmate_position,0.000000,0.000000,0.015615,0.031678,0.125376
9,underpromotion_possibility,0.000000,0.000000,0.000000,0.015617,0.062899


In [4]:
df_mcts = pd.DataFrame({'FEN': fen_names})

for param in range (1, 6):
    lst = []
    for fen in fen_list:
        board = chess.Board(fen)
        start_time = time.time()
        mcts.mcts_findNextMove(game=board, iteration=param)
        execution_time = time.time() - start_time
        lst.append(execution_time)
    df_mcts[f'Number of terations={param}'] = lst


print('MCTS evaluation') # 2mins
df_mcts

MCTS evaluation


,FEN,Number of terations=1,Number of terations=2,Number of terations=3,Number of terations=4,Number of terations=5
0,starting_position,1.004110,2.245153,3.422464,4.336507,5.433907
1,fools_mate,1.129691,2.028559,2.685216,3.789292,4.838996
2,scholars_mate,0.878238,1.994483,2.629188,5.386355,4.015606
3,endgame_king_pawn_vs_king,0.031238,0.046895,0.071538,0.062928,0.266890
4,complex_middle_game,1.336018,2.686586,3.335448,4.237791,4.677894
5,position_with_castling_rights,1.041045,2.228991,2.962809,4.398638,4.038124
6,en_passant_scenario,1.073766,1.991870,3.173079,4.601349,5.598318
7,stalemate_position,0.000000,0.000000,0.000000,0.000000,0.000000
8,checkmate_position,0.386073,0.219601,0.972386,1.083469,1.586881
9,underpromotion_possibility,0.062920,0.125429,0.209577,0.156628,0.219604


2. Average execution time and Move quality

The default depth/iterations is setted to be 3.

We use given dataset from [this site](https://www.kaggle.com/datasets/dev102/chess-fens-evaluations-dataset) for the fen string and use StockFish evaluation from chess-ai.com API for indicating the quality of the move.

In [5]:
#find average time and average score, with default depth/iterations = 3
from app import evaluation

list_of_mode = ['Minimax', 'AlphaBeta', 'MCTS', 'MCTS_NN']

def next_move_generator(board, mode, param=3):
    move = None
    if mode == "Minimax":
        move = minimax_alphabeta.findMoveMinimax(board=board, depth=param)[1]
    elif mode == "AlphaBeta":
        move = minimax_alphabeta.findMoveAlphaBeta(board=board, depth=param, alpha=-float('inf'), beta=float('inf'))[1]
    elif mode == "MCTS":
        move = mcts.mcts_findNextMove(game=board, iteration=param)
    elif mode == "MCTS_NN":
        pass

    return move

# Read the CSV file
file_path = 'FilteredEvals.csv'
dataset_fen = pd.read_csv(file_path)

# Select 100 random rows
df_evaluation = dataset_fen.sample(n=100, random_state=42)  # Use random_state for reproducibility

df_evaluation.drop(columns=['Evaluation'], inplace=True)

for mode in list_of_mode:
    score_lst = []
    time_lst = []
    for fen in df_evaluation['FEN']:
        board = chess.Board(fen)
        start_time = time.time()
        move = next_move_generator(board, mode)
        if move in board.legal_moves:
            execution_time = time.time() - start_time
            board.push(move)
            score = evaluation(board)

        else:
            score = None
        score_lst.append(score)
        time_lst.append(execution_time)
    df_evaluation[mode + '_score'] = score_lst
    df_evaluation[mode + '_time'] = time_lst


df_evaluation.to_csv('evalution_results.csv', index=False)
df_evaluation.describe()


,Minimax_score,Minimax_time,AlphaBeta_score,AlphaBeta_time,MCTS_score,MCTS_time,MCTS_NN_time
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,2.475900,1.868372,2.337300,0.258492,0.523400,1.981360,1.506303
std,22.770384,1.350030,22.834447,0.177358,22.994535,0.826349,0.000000
min,-100.000000,0.000000,-100.000000,0.000000,-100.000000,0.314622,1.506303
25%,-4.542500,0.507869,-4.632500,0.106006,-4.632500,1.292938,1.506303
50%,-0.140000,1.869789,-0.120000,0.250808,-0.115000,1.994672,1.506303
75%,3.565000,2.826230,3.835000,0.376713,3.735000,2.629148,1.506303
max,100.000000,4.633123,100.000000,0.761004,100.000000,3.895854,1.506303


3. Gameplay outcome

In [6]:
df_result = pd.DataFrame(index=list_of_mode, columns=list_of_mode)

df_result

,Minimax,AlphaBeta,MCTS,MCTS_NN
Minimax,NaN,NaN,NaN,NaN
AlphaBeta,NaN,NaN,NaN,NaN
MCTS,NaN,NaN,NaN,NaN
MCTS_NN,NaN,NaN,NaN,NaN


In [7]:
# 1 match = 3 mins
"""
The algorithms were implemented at different times,
and running 10 matches sequentially takes a considerable amount of time.
Instead of using a single for loop to execute everything at once,
I split the process into multiple separate cells, with each line running independently.
This allows me to save time by executing them individually.
"""

def check_game_status(game):
    """Check the game status and return the result in the form of (win, loss, draw) of white."""
    if game.is_checkmate():
        return [1, 0, 0] if not game.turn else [0, 1, 0]
    elif game.is_stalemate():
        return [0, 0, 1]
    elif game.is_insufficient_material():
        return [0, 0, 1]
    elif game.is_fifty_moves():
        return [0, 0, 1]
    elif game.is_fivefold_repetition():
        return [0, 0, 1]
    return None

def match(mode1, mode2): # mode1 vs mode2, mode1 is white
    board = chess.Board()

    while True:
        result = check_game_status(board)
        if result:
            return result

        next_move = next_move_generator(board, mode1 if board.turn else mode2)
        
        if next_move is None:
            print(board.fen(), mode1 if board.turn else mode2)
            return 'Error'
        else:
            if next_move in board.legal_moves:
                board.push(next_move)

def matches_simulator(mode1, mode2, nums_of_matches = 2):
    all_matches_results = [0, 0, 0] # win, loss, draw
    for _ in range(nums_of_matches):
        result = match(mode1, mode2)

        all_matches_results = [all_matches_results[i] + result[i] for i in range(3)]

    return all_matches_results

pairs_of_mode = [[a, b] for a in list_of_mode for b in list_of_mode]

for pairs in pairs_of_mode:
    if pairs[0] != 'MCTS_NN' and pairs[1] != 'MCTS_NN':
        print(pairs)
        df_result.loc[pairs[0], pairs[1]] = matches_simulator(pairs[0], pairs[1], 10)

df_result

['Minimax', 'Minimax']
['Minimax', 'AlphaBeta']
['Minimax', 'MCTS']
['AlphaBeta', 'Minimax']
['AlphaBeta', 'AlphaBeta']
['AlphaBeta', 'MCTS']
['MCTS', 'Minimax']
['MCTS', 'AlphaBeta']
['MCTS', 'MCTS']


,Minimax,AlphaBeta,MCTS,MCTS_NN
Minimax,"[2, 5, 3]","[5, 2, 3]","[10, 0, 0]",NaN
AlphaBeta,"[4, 6, 0]","[2, 3, 5]","[9, 0, 1]",NaN
MCTS,"[0, 10, 0]","[0, 10, 0]","[0, 1, 9]",NaN
MCTS_NN,NaN,NaN,NaN,NaN


In [8]:
# for the remaining result:
for pairs in pairs_of_mode:
    if pairs[0] == 'MCTS_NN' or pairs[1] == 'MCTS_NN':
        print(pairs)
        df_result.loc[pairs[0], pairs[1]] = 'test'#matches_simulator(pairs[0], pairs[1], 10)

df_result

['Minimax', 'MCTS_NN']
['AlphaBeta', 'MCTS_NN']
['MCTS', 'MCTS_NN']
['MCTS_NN', 'Minimax']
['MCTS_NN', 'AlphaBeta']
['MCTS_NN', 'MCTS']
['MCTS_NN', 'MCTS_NN']


,Minimax,AlphaBeta,MCTS,MCTS_NN
Minimax,"[2, 5, 3]","[5, 2, 3]","[10, 0, 0]",test
AlphaBeta,"[4, 6, 0]","[2, 3, 5]","[9, 0, 1]",test
MCTS,"[0, 10, 0]","[0, 10, 0]","[0, 1, 9]",test
MCTS_NN,test,test,test,test
